# Esercizio 3 - Hanks

- Il verbo scelto è **Eat**
- Si estrae dal corpus **Brown** *n* > 200 istanze che contengono il verbo **Eat**
- Si effettua parsing e disambiguazione delle frasi estratte
- Si usano i super sensi di WordNet sugli argomenti (subj e obj nel caso di 2 argomenti) del verbo scelto
- Si aggregano i risultati, si calcolano le frequenze, e si stampano i cluster semantici ottenuti

In [5]:
import nltk
from nltk.corpus import brown
from nltk.wsd import lesk
from nltk.corpus import wordnet as wn
import spacy

ModuleNotFoundError: No module named 'spacy'